# Importando as bibliotecas

In [48]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers import Embedding
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
import string
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Weslley\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Funções que serão usadas durante o projeto

### Esta função apenas lê os dados do arquivo informado e retorna o texto

In [10]:
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

### Neste passo estamos criando uma funcao load_doc para tokenizar os textos
A funcao clean_doc efetua os seguintes tratamentos: 
1. tokeniza o documento usando espaco; 
2. Remove pontuacao de cada token; 
3. Remove caracteres especiais de cada token;
4. Remove stop words (ex.: de, para, do...);
5. Remove tokens cujo tamanho seja <= 1;

In [11]:
def clean_doc(doc):
    
    # Separa as palavras (tokens) sempre que encontrar um espaço em branco
    tokens = doc.split()
    
    # Remove pontuação
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    
    # Remove tokens não alfabéticos
    tokens = [word for word in tokens if word.isalpha()]
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    
    # Remove tokens com um caracter ou menos
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

### Esta função carrega o documento, limpa o texto e remove palavras que não pertencem ao vocabulário

In [12]:
def doc_to_line(filename, vocab):
    
    # Carrega o documento
    doc = load_doc(filename)
    
    # Limpa o texto
    tokens = clean_doc(doc)
    
    # Remove palavras que não pertencem ao vocabulário
    tokens = [w for w in tokens if w in vocab]
    return ' '.join(tokens)

### Esta função é responsável por correr um diretório e carregar todos os arquivos que serão usados

In [13]:
# load all docs in a directory
def process_docs(directory, vocab, is_train):
    
    lines = list()
    
    # Para cada arquivo existente no diretório
    for filename in listdir(directory):
    
        # Verifica se deve carregar os dados de treino ou teste (pula o arquivo que não pertence ao conjunto em questão)
        if is_train and filename.startswith('cv9'):
            continue
        if not is_train and not filename.startswith('cv9'):
            continue
        
        # Seta o diretório do arquivo
        path = directory + '/' + filename
        
        # Carrega os dados
        line = doc_to_line(path, vocab)
        
        # Adiciona aos resultados
        lines.append(line)
        
    return lines

### Esta função salva o vocabulário em disco

In [14]:
def save_list(lines, filename):
    
    # Converte os tokens em string
    data = '\n'.join(lines)
    
    # Salva o vocabulário em um arquivo
    file = open(filename, 'w')
    file.write(data)
    file.close()

# Definindo o vocabulário
Dados em http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz

### Estas funções criam o vocabulário de forma manual (sem uso de bibliotecas específicas)

In [15]:
def create_vocabulary (directory, vocab):
    
    # Para cada arquivo existente no diretório
    for filename in listdir(directory):
        
        # Pula os arquivos que pertencem ao conjunto de teste
        if filename.startswith('cv9'):
            continue
        
        # Prepara o path do arquivo
        path = directory + '/' + filename
        
        # Adiciona texto ao vocabulário
        add_doc_to_vocab(path, vocab)
        
def add_doc_to_vocab(filename, vocab):
    
    # Carrega o documento
    doc = load_doc(filename)
    
    # Limpa o texto
    tokens = clean_doc(doc)
    
    # Atualiza o vocabulário
    vocab.update(tokens)

### Cria o vocabulário

In [16]:
# Nosso vocabulário será armazenado nesta variável
vocab = Counter()

# add all docs to vocab
create_vocabulary('dataset/txt_sentoken/pos', vocab)
create_vocabulary('dataset/txt_sentoken/neg', vocab)

# print the size of the vocab
print('O vocabulário possui {} palavras\n'.format(len(vocab)))

# print the top words in the vocab
print('As palavras mais comumns são:\n\n{}'.format(vocab.most_common(50)))

O vocabulário possui 44276 palavras

As palavras mais comumns são:

[('film', 7983), ('one', 4946), ('movie', 4826), ('like', 3201), ('even', 2262), ('good', 2080), ('time', 2041), ('story', 1907), ('films', 1873), ('would', 1844), ('much', 1824), ('also', 1757), ('characters', 1735), ('get', 1724), ('character', 1703), ('two', 1643), ('first', 1588), ('see', 1557), ('way', 1515), ('well', 1511), ('make', 1418), ('really', 1407), ('little', 1351), ('life', 1334), ('plot', 1288), ('people', 1269), ('could', 1248), ('bad', 1248), ('scene', 1241), ('movies', 1238), ('never', 1201), ('best', 1179), ('new', 1140), ('scenes', 1135), ('man', 1131), ('many', 1130), ('doesnt', 1118), ('know', 1092), ('dont', 1086), ('hes', 1024), ('great', 1014), ('another', 992), ('action', 985), ('love', 977), ('us', 967), ('go', 952), ('director', 948), ('end', 946), ('something', 945), ('still', 936)]


### Agora vamos atualizar nosso vocabulario e manter apenas os tokens com mais de 1 ocorrencia

In [17]:
min_occurane = 2
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print('O novo vocabulário possui {} palavras'.format(len(tokens)))

O novo vocabulário possui 25767 palavras


### Finalmente vamos salvar nosso vocabulario em disco

In [18]:
# Salva o vocabulário em um arquivo
save_list(tokens, 'dataset/txt_sentoken/vocab.txt')

# Carregando os dados de treino

In [19]:
# Carrega o vocabulário
vocab_filename = 'dataset/txt_sentoken/vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

In [20]:
# Carrega os dados de treino
positive_lines = process_docs('dataset/txt_sentoken/pos', vocab, True)
negative_lines = process_docs('dataset/txt_sentoken/neg', vocab, True)
docs_train = negative_lines + positive_lines

# Seta a variável target (sabemos que os primeiros 900 textos são comentários negativos e os outros 900 são positivos)
ytrain = np.array([0 for _ in range(900)] + [1 for _ in range(900)])

# Exibe um exemplo
print(len(positive_lines), len(negative_lines))
positive_lines[1]

900 900


'every movie comes along suspect studio every indication stinker everybodys surprise perhaps even studio film becomes critical darling mtv films election high school comedy starring matthew broderick reese witherspoon current example anybody know film existed week opened plot deceptively simple george washington carver high school student elections tracy flick reese witherspoon overachiever hand raised nearly every question way way high mr matthew broderick sick megalomaniac student encourages paul jock run pauls nihilistic sister jumps race well personal reasons dark side sleeper success expectations low going fact quality stuff made reviews even enthusiastic right cant help going baggage glowing reviews contrast negative baggage reviewers likely election good film live hype makes election disappointing contains significant plot details lifted directly rushmore released months earlier similarities staggering tracy flick election president extraordinary number clubs involved school pla

# Carregando os dados de teste

In [21]:
# load all test reviews
positive_lines = process_docs('dataset/txt_sentoken/pos', vocab, False)
negative_lines = process_docs('dataset/txt_sentoken/neg', vocab, False)
docs_test = negative_lines + positive_lines

# Seta a variável target (sabemos que os primeiros 100 textos são comentários negativos e os outros 100 são positivos)
ytest = np.array([0 for _ in range(100)] + [1 for _ in range(100)]) # os primeiros sempre sao da classe pos.

# Exibe um exemplo
print(len(positive_lines), len(negative_lines))
positive_lines[1]

100 100


'start movie reminded parts movie stargate people looking around egyptian temple reading dangerous thing going destroy earth future sort confusing bit involving fakelooking cyborg things movie jumps future movie improves leaps bounds basic idea behind movie every make every years evil force comes destroy earth things needed defend menace four elements nature plus fifth element plot movie really isnt important thing though movie good special effects part music background fits mood well bruce willis illegal driver futuristic new york city one day lady bandages drops trunk movie happens plot twists interesting movie never fails present viewer variety different locations also fair bit action film particularly towards end characters plain strange including deejay drag bruce willis normal job blowing things away like always movie definitely watchable rarely slows one scifi films youll saying cool followed hell give fifth element'

# Resumo
Até este ponto fizemos os seguintes passos:
* Limpeza dos dados
* Criação de um vocabulário

Agora já é possível receber um texto qualquer e, por meio das funções que criamos, fazer a limpeza deste texto e restringi-lo ao vocabulário que criamos com os dados de teste. Daqui pra frente vamos explorar diferentes maneiras de transformar o nosso texto (que já está limpo) e uma **representação numérica**. 

# Criando o modelo com embedding layer

### Transforma texto em números

In [22]:
# Tamanho do vocabulário
vocab_size = len(vocab)

# Cria e treina o tokenizer
tokenizer = Tokenizer(num_words= vocab_size, filters='')
tokenizer.fit_on_texts(docs_train)

# Transforma os dados de treino
encoded_docs = tokenizer.texts_to_sequences(docs_train)

### Usa um vetor de mesmo tamanho para representar todos os textos (completa os vetores menores com zeros)

In [23]:
# pad documents to a max length of 4 words
max_length = 0
for enc in encoded_docs:
    max_length = max(max_length, len(enc))

padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

### Define e compila o modelo

In [55]:
EMBEDDING_DIM = 8

model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length=max_length))
model.add(Flatten())
model.add(Dense(50, activation='sigmoid'))
model.add(Dropout(rate=0.2))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# summarize the model
print(model.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 1317, 8)           206136    
_________________________________________________________________
flatten_7 (Flatten)          (None, 10536)             0         
_________________________________________________________________
dense_14 (Dense)             (None, 50)                526850    
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 51        
Total params: 733,037
Trainable params: 733,037
Non-trainable params: 0
_________________________________________________________________
None


### Treina o modelo

In [56]:
model.fit(padded_docs, ytrain, epochs=100, verbose=1, validation_split=0.25)

Train on 1350 samples, validate on 450 samples
Epoch 1/100
1350/1350 [==============================] - 1s 1ms/sample - loss: 0.6556 - acc: 0.6556 - val_loss: 1.0979 - val_acc: 0.0000e+00
Epoch 2/100
1350/1350 [==============================] - 1s 660us/sample - loss: 0.6015 - acc: 0.6815 - val_loss: 1.2183 - val_acc: 0.0000e+00
Epoch 3/100
1350/1350 [==============================] - 1s 648us/sample - loss: 0.5023 - acc: 0.7400 - val_loss: 1.1498 - val_acc: 0.0000e+00
Epoch 4/100
1350/1350 [==============================] - 1s 648us/sample - loss: 0.3039 - acc: 0.9348 - val_loss: 1.1994 - val_acc: 0.0133
Epoch 5/100
1350/1350 [==============================] - 1s 678us/sample - loss: 0.1397 - acc: 0.9919 - val_loss: 1.1222 - val_acc: 0.0822
Epoch 6/100
1350/1350 [==============================] - 1s 648us/sample - loss: 0.0663 - acc: 1.0000 - val_loss: 1.2962 - val_acc: 0.0756
Epoch 7/100
1350/1350 [==============================] - 1s 666us/sample - loss: 0.0388 - acc: 1.0000 - val_l

1350/1350 [==============================] - 1s 671us/sample - loss: 4.4894e-04 - acc: 1.0000 - val_loss: 2.0307 - val_acc: 0.2533
Epoch 59/100
1350/1350 [==============================] - 1s 662us/sample - loss: 4.2888e-04 - acc: 1.0000 - val_loss: 2.0346 - val_acc: 0.2556
Epoch 60/100
1350/1350 [==============================] - 1s 683us/sample - loss: 4.4621e-04 - acc: 1.0000 - val_loss: 2.0455 - val_acc: 0.2533
Epoch 61/100
1350/1350 [==============================] - 1s 766us/sample - loss: 4.1225e-04 - acc: 1.0000 - val_loss: 2.0580 - val_acc: 0.2511
Epoch 62/100
1350/1350 [==============================] - 1s 828us/sample - loss: 3.8554e-04 - acc: 1.0000 - val_loss: 2.0740 - val_acc: 0.2511
Epoch 63/100
1350/1350 [==============================] - 1s 799us/sample - loss: 4.0931e-04 - acc: 1.0000 - val_loss: 2.0746 - val_acc: 0.2556
Epoch 64/100
1350/1350 [==============================] - 1s 734us/sample - loss: 3.5226e-04 - acc: 1.0000 - val_loss: 2.0884 - val_acc: 0.2556
Epoch

### Avalia o modelo nos dados de teste

In [57]:
# Transforma strings em números
encoded_docs_test = tokenizer.texts_to_sequences(docs_test)

# Preenche os vetores menores com zeros
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')

# Avalia o modelo nos dados de teste
loss, accuracy = model.evaluate(padded_docs_test, ytest, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 62.000000


# Referências
* https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
* https://www.tensorflow.org/hub
* https://tfhub.dev/google/nnlm-en-dim128/2
* https://machinelearningmastery.com/deep-learning-bag-of-words-model-sentiment-analysis/